In [2]:
# Importar las bibliotecas necesarias
import pandas as pd
from transformers import pipeline
from datetime import datetime

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import torch

# Cargar el modelo guardado desde la ruta completa
model_path = r"C:\Users\Lucas\Desktop\Analisis de sentimientos\Models\trading_model.pt"
model = torch.load(model_path)

# Asegúrate de poner el modelo en modo de evaluación (si es para inferencia)
model.eval()

# Imprimir el modelo para verificar que se cargó correctamente
print(model)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_23100\628218295.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


# ----------- LIMPIEZA DE DATOS ------------

In [41]:
import pandas as pd
import numpy as np
from transformers import pipeline

In [86]:
# Rutas de los archivos CSV
items_csv_path = r"C:\Users\Lucas\Desktop\Analisis de sentimientos\Data\DataPhone\20191226-items.csv"
reviews_csv_path = r"C:\Users\Lucas\Desktop\Analisis de sentimientos\Data\DataPhone\20191226-reviews.csv"

# Cargar datos desde los archivos CSV
print("Cargando datos...")
items_df = pd.read_csv(items_csv_path)
reviews_df = pd.read_csv(reviews_csv_path)

Cargando datos...


In [87]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   asin           720 non-null    object 
 1   brand          716 non-null    object 
 2   title          720 non-null    object 
 3   url            720 non-null    object 
 4   image          720 non-null    object 
 5   rating         720 non-null    float64
 6   reviewUrl      720 non-null    object 
 7   totalReviews   720 non-null    int64  
 8   price          720 non-null    float64
 9   originalPrice  720 non-null    float64
dtypes: float64(3), int64(1), object(6)
memory usage: 56.4+ KB


In [88]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67986 entries, 0 to 67985
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   asin          67986 non-null  object 
 1   name          67983 non-null  object 
 2   rating        67986 non-null  int64  
 3   date          67986 non-null  object 
 4   verified      67986 non-null  bool   
 5   title         67957 non-null  object 
 6   body          67960 non-null  object 
 7   helpfulVotes  27215 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 3.7+ MB


In [90]:
from transformers import pipeline

# Cargar el modelo de análisis de sentimientos
sentiment_analyzer = pipeline("sentiment-analysis", truncation=True, padding=True, max_length=128)

# Aplicar el análisis de sentimientos directamente sobre la columna 'body' y asignar a una nueva columna 'sentiment'
reviews_df['sentiment'] = reviews_df['body'].apply(
    lambda x: sentiment_analyzer(x)[0]['label'] if pd.notnull(x) else 'Neutral'
)

# Mapear los resultados del modelo a 'Positive' o 'Negative'
reviews_df['sentiment'] = reviews_df['sentiment'].map({'POSITIVE': 'Positive', 'NEGATIVE': 'Negative'})

# Verificar los primeros resultados
print(reviews_df[['title', 'body', 'sentiment']].head())



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


                                          title  \
0                   Def not best, but not worst   
1                   Text Messaging Doesn't Work   
2                               Love This Phone   
3                       Love the Phone, BUT...!   
4  Great phone service and options, lousy case!   

                                                body sentiment  
0  I had the Samsung A600 for awhile which is abs...  Negative  
1  Due to a software issue between Nokia and Spri...  Negative  
2  This is a great, reliable phone. I also purcha...  Negative  
3  I love the phone and all, because I really did...  Positive  
4  The phone has been great for every purpose it ...  Negative  


In [84]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67986 entries, 0 to 67985
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   asin          67986 non-null  object 
 1   name          67983 non-null  object 
 2   rating        67986 non-null  int64  
 3   date          67986 non-null  object 
 4   verified      67986 non-null  bool   
 5   title         67957 non-null  object 
 6   body          67960 non-null  object 
 7   helpfulVotes  27215 non-null  float64
 8   sentiment     67960 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 4.2+ MB


In [85]:
# Ruta para guardar el DataFrame combinado
save_path = r"C:\Users\Lucas\Desktop\Analisis de sentimientos\Data\DataPhone\reviews_sentiment.csv"

# Guardar el DataFrame combinado en un archivo CSV
reviews_df.to_csv(save_path, index=False)

# Confirmación de que el archivo ha sido guardado
print(f"El archivo final ha sido guardado en: {save_path}")

El archivo final ha sido guardado en: C:\Users\Lucas\Desktop\Analisis de sentimientos\Data\DataPhone\reviews_sentiment.csv
